TACOTRON

In [1]:
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd
from mxnet import gluon
from IPython.display import clear_output

In [2]:
ctx= mx.cpu()
num_hidden = 256
K=16
emb_size=256;

data = mx.sym.Variable('data')
emb = mx.sym.Embedding(data=data, input_dim=100, output_dim=emb_size)

In [28]:

def prenet_pass(data):
    prenet = gluon.nn.Sequential() #consider to use HybridSequential
    with prenet.name_scope():
        prenet.add(gluon.nn.Dense(num_hidden,activation="relu"))
        prenet.add(gluon.nn.Dropout(0.5)) #how to use the condition "if train" same as TF?
        prenet.add(gluon.nn.Dense(num_hidden//2,activation="relu"))
        prenet.add(gluon.nn.Dropout(0.5))
    prenet.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=ctx)
    return prenet(data)

In [29]:
#banco di filtri convolutivi. Vengono creati K filtri con kernel 1D di dimensione:k 
def conv1dBank(conv_input):
    conv=mx.sym.Convolution(data=conv_input, kernel=(1,1), num_filter=emb_size//2)
    (conv, mean, var) = mx.sym.BatchNorm(data=conv, output_mean_var=True)
    conv = mx.sym.Activation(data=conv, act_type='relu')
    for k in range(2, K+1):
        convi = mx.sym.Convolution(data=conv_input, kernel=(k,1), num_filter=emb_size//2)
        (convi, mean, var) = mx.sym.BatchNorm(data=convi, output_mean_var=True)
        convi = mx.sym.Activation(data=convi, act_type='relu')
        conv = mx.symbol.concat(conv,convi)
    return conv

convbank=conv1dBank(emb)
#mx.viz.plot_network(convbank)
#se si usa infer_shape su convbank dando la dimensione dell'input, viene dedotta la shape appunto 

In [30]:
#highway
def highway_layer(data):
    H= mx.symbol.Activation(data=mx.symbol.FullyConnected(data=data, num_hidden=emb_size//2), act_type="relu")
    T= mx.symbol.Activation(data=mx.symbol.FullyConnected(data=data, num_hidden=emb_size//2, bias=mx.sym.Variable('bias') ), act_type="sigmoid")
    return  H * T + data * (1.0 - T)


In [42]:
def BidirectionalGRULayer(data):
    net = mx.sym.RNN(data=data,bidirectional=True,mode='gru', num_layers=1, state_size=emb_size//2)
    return net

In [44]:
#CBHG
def CBHG(data):
    prenetted_data = prenet_pass(data)
    bank = conv1dBank(prenetted_data)
    poold_bank = mx.sym.Pooling(data=bank, pool_type='max', kernel=(2, 1), stride=(1,1)) #TOSTUDY: stride?

    proj1 = mx.sym.Convolution(data=poold_bank, kernel=(3,1), num_filter=emb_size//2)
    (proj1, proj1_mean, proj1_var) = mx.sym.BatchNorm(data=proj1, output_mean_var=True) #TOSTUDY: does the symbol encapsule the mean and var too?
    proj1 = mx.sym.Activation(data=proj1, act_type='relu')#Is it ok to declare/use again the same variable?  

    proj2 = mx.sym.Convolution(proj1, kernel=(3,1), num_filter=emb_size//2)
    (proj2, proj2_mean, proj2_var) = mx.sym.BatchNorm(data=proj2, output_mean_var=True)
    residual= proj2 + emb

    for i in range(4):
        residual = highway(residual)
    highway_pass = residual
   
    encoder = BidirectionalGRULayer(highway_pass)

    return encoder

#mx.viz.plot_network(CBHG(emb) )